In [1]:
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np


In [22]:
df = pd.read_csv('output_file_1_1000.csv')
# data.info()

In [3]:
    # id: A unique identifier for each row in the dataset.
    # event_id: An identifier for the event or match.
    # innings: This denotes the inning of the match. In cricket, there are typically 2 innings.
    # overs: This refers to the number of overs bowled in the match so far. An over in cricket consists of six deliveries.
    # ball_no: This could be the number of the ball in the current over.
    # match_ball_no: This could be the total number of balls delivered in the match so far.
    # innings_runs: This indicates the total number of runs scored in the current innings so far.
    # innings_wickets: This could be the total number of wickets fallen in the current innings so far.
    # innings_target: The target set for the team batting in the second innings.
    # innings_remaining_runs: The remaining runs the batting side needs to win, usually applicable to the second innings.
    # dismissal_dismissal: A boolean (True/False) value indicating if a dismissal took place on this ball.
    # dismissal_bowled: A boolean (True/False) value indicating if the dismissal was by being bowled.
    # dismissal_minutes: Not immediately clear, this could possibly refer to the time taken for a dismissal decision or the time at which the dismissal occurred.
    # dismissal_bowler_id, dismissal_bowler_name: The ID and name of the bowler who took the wicket.
    # dismissal_batsman_id, dismissal_batsman_name: The ID and name of the batsman who got dismissed.

In [4]:
    # innings_remaining_balls: The remaining balls the batting side needs to face, usually applicable to the second innings.
    # run_rate_required: The required run rate for the batting side to win the match.
    # bowler_id: The ID of the bowler.
    # batter_id: The ID of the batsman.
    # batter_balls_faced: The number of balls the batsman has faced.
    # batter_runs: The number of runs the batsman has scored.
    # nonstriker_id: The ID of the non-striker batsman.
    # nonstriker_balls_faced: The number of balls the non-striker has faced.
    # nonstriker_runs: The number of runs the non-striker has scored.
    # outcome: The outcome of the delivery.
    # wickets_lost: The number of wickets lost in the current over.
    # wicket_how: The manner in which the wicket fell.
    # date: The date of the match.
    # text: Possibly a textual description of the delivery.
    # short_text: Possibly a short textual description of the delivery.
    # home_score: The score of the home team.
    # away_score: The score of the away team.
    # score_value: Not immediately clear, this could possibly refer to the number of runs scored in the delivery.
    # sequence: Not immediately clear, this could possibly refer to the sequence of events in the match.
    # bbb_timestamp: The timestamp of the ball by ball event.

In [5]:

# # Separate numerical and categorical columns
# numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
# categorical_cols = df.select_dtypes(include=['object', 'bool']).columns

# # Fill missing values in numeric columns with the column median
# for col in numeric_cols:
#     df[col].fillna(df[col].median(), inplace=True)

# # Fill missing values in categorical columns with the most frequent category
# for col in categorical_cols:
#     df[col].fillna(df[col].mode()[0], inplace=True)
# print(df.isnull().sum().sum())

In [6]:
#     Total runs scored by batter: Sum up 'batter_runs' for each ball faced by the batter.
#     Strike rate of batter: Total runs scored divided by total balls faced, multiplied by 100.
#     Average of batter: Total runs scored divided by total number of times out.
#     Max score of batter: For each innings (innings_id), sum up 'batter_runs', then find the maximum of these sums.
#     Number of 4s scored by batter: Count the number of balls where 'batter_runs' equals 4.
#     Number of 6s scored by batter: Count the number of balls where 'batter_runs' equals 6.
#     Number of 50s scored by batter: For each innings (innings_id), sum up 'batter_runs', then count the number of sums that reach or exceed 50 but are less than 100.
#     Number of 100s scored by batter: For each innings (innings_id), sum up 'batter_runs', then count the number of sums that reach or exceed 100.

# For each bowler:

#     Innings: Count the unique number of 'innings_id' for the given bowler.
#     Total runs conceded by the bowler: Sum up 'bowler_conceded' for each ball bowled by the bowler.
#     Total wickets taken by the bowler: Count the number of balls where the bowler's ID appears in 'dismissal_bowler_id' and 'outcome' is 'out'.
#     Maximum number of wickets taken by bowler in 1 match: For each match (event_id), count the number of wickets taken by the bowler (each ball where the bowler's ID appears in 'dismissal_bowler_id' and 'outcome' is 'out'), then find the maximum of these counts.
#     Economy of the bowler: Total runs conceded divided by total overs bowled. To calculate total overs bowled, divide the total balls bowled by 6.
#     Average of the bowler: Total runs conceded divided by total wickets taken.
#     Number of 3 wickets haul taken by bowler: For each innings (innings_id), count the number of wickets taken by the bowler, then count the number of innings where this count is 3 or more.
#     Number of 5 wickets haul taken by bowler: Similarly, for each innings (innings_id), count the number of wickets taken by the bowler, then count the number of innings where this count is 5 or more.



In [ ]:
# Updated code for batsman stats
batsman_runs = df.groupby(['batter_id', 'event_id', 'innings_id'])['batter_runs'].max().groupby('batter_id').sum()

batsman_balls = df.groupby(['batter_id', 'event_id', 'innings_id'])['batter_balls_faced'].max().groupby('batter_id').sum()

batsman_outs = df[df['outcome'] == 'out'].groupby('batter_id').size()

batsman_max_score = df.groupby(['batter_id', 'event_id', 'innings_id'])['batter_runs'].max().groupby('batter_id').max()

batsman_fours = df[df['outcome'] == 'four'].groupby(['batter_id', 'event_id', 'innings_id']).size().groupby('batter_id').sum()

batsman_sixes = df[df['outcome'] == 'six'].groupby(['batter_id', 'event_id', 'innings_id']).size().groupby('batter_id').sum()

batsman_fifties = df.groupby(['batter_id', 'event_id', 'innings_id'])['batter_runs'].max().groupby('batter_id').apply(lambda x: sum((x >= 50) & (x < 100)))

batsman_hundreds = df.groupby(['batter_id', 'event_id', 'innings_id'])['batter_runs'].max().groupby('batter_id').apply(lambda x: sum(x >= 100))

batsman_stats = pd.concat([batsman_runs, batsman_balls, batsman_outs, batsman_max_score, batsman_fours, batsman_sixes, batsman_fifties, batsman_hundreds], axis=1)

batsman_stats.columns = ['total_runs', 'total_balls', 'times_out', 'max_score', 'num_fours', 'num_sixes', 'num_fifties', 'num_hundreds']

batsman_stats['average'] = batsman_stats['total_runs'] / batsman_stats['times_out'].replace(0,1)

batsman_stats['strike_rate'] = batsman_stats['total_runs'] / batsman_stats['total_balls'] * 100

# Updated code for bowler stats
bowler_innings = df.groupby('bowler_id')['innings_id'].nunique()

bowler_runs_conceded = df.groupby(['bowler_id', 'event_id', 'innings_id'])['bowler_conceded'].max().groupby('bowler_id').sum()

bowler_wickets = df[df['outcome'] == 'out'].groupby('bowler_id').size()

bowler_max_wickets = df[df['outcome'] == 'out'].groupby(['bowler_id', 'event_id', 'innings_id']).size().groupby('bowler_id').max()

bowler_overs = df.groupby(['bowler_id', 'event_id', 'innings_id'])['bowler_overs'].max().groupby('bowler_id').sum()

bowler_three_wickets = df[df['outcome'] == 'out'].groupby(['bowler_id', 'event_id', 'innings_id']).size().groupby('bowler_id').apply(lambda x: sum(x >= 3))

bowler_five_wickets = df[df['outcome'] == 'out'].groupby(['bowler_id', 'event_id', 'innings_id']).size().groupby('bowler_id').apply(lambda x: sum(x >= 5))

bowler_stats = pd.concat([bowler_innings, bowler_runs_conceded, bowler_wickets, bowler_max_wickets, bowler_overs, bowler_three_wickets, bowler_five_wickets], axis=1)

bowler_stats.columns = ['innings', 'total_runs_conceded', 'total_wickets', 'max_wickets_in_match', 'total_overs', 'num_three_wickets', 'num_five_wickets']

bowler_stats['average'] = bowler_stats['total_runs_conceded'] / bowler_stats['total_wickets']

bowler_stats['economy'] = bowler_stats['total_runs_conceded'] / bowler_stats['total_overs']

batsman_stats, bowler_stats


In [43]:
batsman_stats

,total_runs,total_balls,times_out,max_score,num_fours,num_sixes,num_fifties,num_hundreds,average,strike_rate
batter_id,,,,,,,,,,
4578.0,0.0,0.0,NaN,0.0,NaN,NaN,0,0,NaN,NaN
5390.0,1.0,2.0,1.0,1.0,NaN,NaN,0,0,1.0,50.000000
5702.0,17.0,21.0,1.0,17.0,2.0,NaN,0,0,17.0,80.952381
6128.0,27.0,14.0,NaN,27.0,4.0,1.0,0,0,NaN,192.857143
6513.0,110.0,65.0,1.0,97.0,7.0,5.0,1,0,110.0,169.230769
...,...,...,...,...,...,...,...,...,...,...
52057.0,14.0,18.0,NaN,14.0,1.0,NaN,0,0,NaN,77.777778
52445.0,4.0,9.0,1.0,4.0,NaN,NaN,0,0,4.0,44.444444
52917.0,9.0,4.0,1.0,9.0,2.0,NaN,0,0,9.0,225.000000


In [ ]:
# First, let's normalize the metrics between 0 and 1
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# Normalize the batsman statistics
batsman_stats_normalized = pd.DataFrame(scaler.fit_transform(batsman_stats), columns=batsman_stats.columns, index=batsman_stats.index)

# Normalize the bowler statistics
bowler_stats_normalized = pd.DataFrame(scaler.fit_transform(bowler_stats), columns=bowler_stats.columns, index=bowler_stats.index)

# Calculate the overall rating for batters and bowlers as the mean of the normalized metrics
batsman_stats_normalized['rating'] = batsman_stats_normalized.mean(axis=1)
bowler_stats_normalized['rating'] = bowler_stats_normalized.mean(axis=1)

batsman_stats_normalized, bowler_stats_normalized


In [28]:
batsman_stats_normalized.head(5)

,total_runs,total_balls,times_out,max_score,num_fours,num_sixes,num_fifties,num_hundreds,average,strike_rate,rating
batter_id,,,,,,,,,,,
4578.0,0.000000,0.000000,NaN,0.000000,NaN,NaN,0.0,0.0,NaN,NaN,0.000000
5390.0,0.007874,0.019048,0.0,0.010309,NaN,NaN,0.0,0.0,0.009091,0.222222,0.033568
5702.0,0.133858,0.200000,0.0,0.175258,0.0625,NaN,0.0,0.0,0.154545,0.359788,0.120661
6128.0,0.212598,0.133333,NaN,0.278351,0.1875,0.0,0.0,0.0,NaN,0.857143,0.208616
6513.0,0.866142,0.619048,0.0,1.000000,0.3750,1.0,1.0,0.0,1.000000,0.752137,0.661233


In [ ]:
# Define the weights for the batters
batter_weights = {
    "total_runs": 0.3,
    "strike_rate": 0.3,
    "average": 0.2,
    "max_score": 0.2
}

# Define the weights for the bowlers
bowler_weights = {
    "total_wickets": 0.3,
    "economy": 0.3,
    "average": 0.2,
    "max_wickets_in_match": 0.2
}

# Compute the weighted ratings
batsman_stats_normalized["weighted_rating"] = batsman_stats_normalized[batter_weights.keys()].apply(lambda row: np.sum(row * list(batter_weights.values())), axis=1)
bowler_stats_normalized["weighted_rating"] = bowler_stats_normalized[bowler_weights.keys()].apply(lambda row: np.sum(row * list(bowler_weights.values())), axis=1)

batsman_stats_normalized, bowler_stats_normalized


In [ ]:

# Batsman Strengths
batsman_strengths = pd.DataFrame(index=batsman_stats.index)
batsman_strengths['strike_rate'] = batsman_stats['total_runs'] / batsman_stats['total_balls'] * 100
batsman_strengths['average'] = batsman_stats['total_runs'] / batsman_stats['times_out']

# For performance under pressure, let's assume that "pressure" is defined as chasing a large total
df_pressure = df[df['innings_remaining_runs'] > 30]
batsman_strengths['performance_under_pressure'] = df_pressure.groupby('batter_id')['batter_runs'].sum() / df_pressure.groupby('batter_id').size() * 100

# Batsman Weaknesses
batsman_weaknesses = pd.DataFrame(index=batsman_stats.index)
batsman_weaknesses['lbw_dismissals'] = df[df['wicket_how'] == 'lbw'].groupby('batter_id').size() / batsman_stats['times_out']
batsman_weaknesses['caught_dismissals'] = df[df['wicket_how'] == 'caught'].groupby('batter_id').size() / batsman_stats['times_out']

df_pressure = df[df['innings_remaining_runs'] > 30]
batsman_weaknesses['poor_performance_under_pressure'] = 100 - (df_pressure.groupby('batter_id')['batter_runs'].sum() / df_pressure.groupby('batter_id').size() * 100)

# Bowler Strengths
bowler_strengths = pd.DataFrame(index=bowler_stats.index)
bowler_strengths['wickets_per_match'] = bowler_stats['total_wickets'] / bowler_stats['innings']
bowler_strengths['economy_rate'] = bowler_stats['total_runs_conceded'] / bowler_stats['total_overs']

df_powerplay = df[df['overs'] <= 6]
bowler_strengths['effectiveness_in_powerplay'] = df_powerplay[df_powerplay['outcome'] == 'out'].groupby('bowler_id').size() / df_powerplay.groupby('bowler_id').size()

df_pressure = df[df['overs'] > 15]
bowler_strengths['performance_under_pressure'] = df_pressure[df_pressure['outcome'] == 'out'].groupby('bowler_id').size() / df_pressure.groupby('bowler_id').size()

# Bowler Weaknesses
bowler_weaknesses = pd.DataFrame(index=bowler_stats.index)
bowler_weaknesses['boundaries_conceded'] = df[(df['batter_runs'] == 4) | (df['batter_runs'] == 6)].groupby('bowler_id').size() / df.groupby('bowler_id').size()

df_aggressive = df[df['batter_runs'] >= 4]
bowler_weaknesses['struggles_against_aggressive_batsmen'] = df_aggressive.groupby('bowler_id')['batter_runs'].sum() / df_aggressive.groupby('bowler_id').size() * 6

bowler_weaknesses['ineffectiveness_in_powerplay'] = 1 - bowler_strengths['effectiveness_in_powerplay']
bowler_weaknesses['poor_performance_under_pressure'] = 1 - bowler_strengths['performance_under_pressure']

batsman_strengths, batsman_weaknesses, bowler_strengths, bowler_weaknesses
